# NEMO examples

In a clean directory we will install NEMO, with a non-graphics device driver. We will create and integrate a Plummer sphere, and manually compile a NEMO program.

This first cell will re-install a clean version of the source code, from github. Since the code is open source, you will not need an account on github, but the **git** command needs to be present.

# Installing NEMO

In [1]:
cd $HOME
mkdir -p NEMO
cd NEMO
mkdir -p demo
cd demo
rm -rf nemo_demo
git clone https://github.com/teuben/nemo nemo_demo

Cloning into 'nemo_demo'...
remote: Enumerating objects: 82683, done.
remote: Counting objects: 100% (7587/7587), done.
remote: Compressing objects: 100% (2605/2605), done.
remote: Total 82683 (delta 5155), reused 7369 (delta 4945), pack-reused 75096
Receiving objects: 100% (82683/82683), 43.86 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (59369/59369), done.


The next cell can always be revisited to redo a configure and build. But we show the version of NEMO just to make sure we are in the right directory

In [2]:
cd $HOME/NEMO/demo/nemo_demo
cat VERSION

4.4.3


Now we can configure the package.  We configure with a simple non-graphics YAPP graphics driver.  YAPP stands for *Yet Another Plotting Package*.  We pick the "ps" (postscript) driver.   There is also an "svg" (in development) which might be more fun to display actual results, though in practice the pgplot driver is used.

In [3]:
date0=$(date)

./configure --with-yapp=ps


checking NEMO config... checking build system type... x86_64-unknown-linux-gnu
checking host system type... x86_64-unknown-linux-gnu
Found CANONICAL_HOST = linux-gnu or x86_64-unknown-linux-gnu or x86_64
Found CANONICAL_BUILD = x86_64-unknown-linux-gnu
checking target system type... x86_64-unknown-linux-gnu
Found CANONICAL_TARGET = x86_64-unknown-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C... yes
checking whether gcc accepts -g... yes
checking for gcc option to enable C11 features... none needed
checking for stdio.h... yes
checking for stdlib.h... yes
checking for string.h... yes
checking for inttypes.h... yes
checking for stdint.h... yes
checking for strings.h... yes
checking for sys/stat.h... yes
checking for sys/types.h... 

The next step is building the software (libraries and binaries) in place. NEMO does normally not install itself in some /usr/local/bin type directory, the build is done in-place. This has some impact how users can use NEMO. More about that later.

The **make build** will do a basic install:

In [4]:
make build

(source nemo_start.sh ; make postconfig mknemos)
make[1]: Entering directory '/home/teuben/NEMO/demo/nemo_demo'
Checking directory structure on Wed Jan 31 08:33:36 PM EST 2024
By teuben on k2
No tar symlink
Updating /home/teuben/NEMO/demo/nemo_demo/adm/TIMESTAMP file on Wed Jan 31 08:33:36 PM EST 2024
Creating /home/teuben/NEMO/demo/nemo_demo/etc/motd file on Wed Jan 31 08:33:36 PM EST 2024
Creating /home/teuben/NEMO/demo/nemo_demo/nemo_local.csh file on Wed Jan 31 08:33:36 PM EST 2024
Creating /home/teuben/NEMO/demo/nemo_demo/nemo_local.sh file on Wed Jan 31 08:33:36 PM EST 2024
Done with dirs at Wed Jan 31 08:33:36 PM EST 2024
cp config.h makedefs /home/teuben/NEMO/demo/nemo_demo/lib
Installing scripts on Wed Jan 31 08:33:36 PM EST 2024
By teuben on k2
(cd /home/teuben/NEMO/demo/nemo_demo/src/scripts; make install)
make[2]: Entering directory '/home/teuben/NEMO/demo/nemo_demo/src/scripts'
cp Makefile.lib /home/teuben/NEMO/demo/nemo_demo/lib
cp mknemo mkpdoc manlaser nemoman nemobench

Check if the build was ok. A number of tests produce a data file which is regressed against a known answer. If these fail, you would see lines starting with **BSF** on the screen. 

The **make check** will do this:

In [5]:
make check


(source nemo_start.sh; src/scripts/testsuite | tee -a install.log )
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/image/fits
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/image/io
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/image/misc
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/image/rotcur
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/image/trans
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/image/wcs
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/kernel/fortran
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/kernel/io
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/kernel/misc
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/kernel/tab
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/nbody/cores
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/nbody/evolve/aarseth/nbody0
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/nbody/evolve/aarseth/tools
TESTSUITE: OK   /home/teuben/NEMO/demo/nemo_demo/src/nbody/evo

And finally a benchmark to check how the speed compares with an otherwise random 2020-type laptop. Currently 6 nbody codes are run, each of them typically taking 5 seconds to compute.

The **make bench5** will do this:

In [6]:
make bench5


(source nemo_start.sh; /usr/bin/time src/scripts/nemo.bench mode=5 | tee -a install.log )
NEMOBENCH: (2022-09-14) bench362389 : nbody0=3072 nbody1=10240 nbody3=1000000 size=256 ften=1 clean=1 bsf=0 tmp=bench362389
k2
Linux k2 6.2.0-35-generic #35~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Oct 6 10:23:26 UTC 2 x86_64 x86_64 x86_64 GNU/Linux
Wed Jan 31 08:35:15 PM EST 2024
Equal 5sec CPU time per code, no output files to check
Codes:  directcode [mkplummer] gyrfalcON hackcode1 [mkorbit] orbint potcode treecode1
Tstop: 4.28*1 5.64*1 7.89*1 787400.0*1 6.66*1 3.00*1 [5sec CPU for each on a 11th Gen Intel(R) Core(TM) i5-1135G7 up to 4.2GHz using gcc-11]
.1.23.456.
CPU_USAGE  directcode  :  4.66  4.65  0.00  0.00  0.00  1730299965
CPU_USAGE  gyrfalcON   :  4.60  4.59  0.00  0.00  0.00  1730300432
CPU_USAGE  hackcode1   :  4.90  4.90  0.00  0.00  0.00  1730300892
CPU_USAGE  orbint      :  4.58  4.56  0.00  0.00  0.00  1730301382
CPU_USAGE  potcode     :  4.98  4.97  0.00  0.00  0.00  1730301840
CP

# Using NEMO
Now the package is ready for anybody to use. For this a user needs to source the **nemo_start.sh** file in their shell before NEMO commands are available. 

In our case here:


In [7]:
source $HOME/NEMO/demo/nemo_demo/nemo_start.sh

and now the **nemo** command is available to check your active version of NEMO:

In [8]:
nemo

NEMO:        /home/teuben/NEMO/demo/nemo_demo  - Version:4.4.3
git:         Branch:master     Counter:11692      Date: 2024/01/30_15:11:00
python:      /home/teuben/anaconda3/bin/python  - Python 3.10.9
OS_release:  Linux Description:	Ubuntu 22.04.3 LTS


## A Plummer sphere

To create a Plummer (1911) sphere with 128 particles the **mkplummer** program is available.   

First a few ways to get help on this program


In [9]:
mkplummer help=h

out              : Output file name [???]
nbody            : Number of particles [???]
mlow             : Low mass fraction cutoff of Plummer dist [0]
mfrac            : Mass fraction used of Plummer distribution [0.999]
rfrac            : Radius fraction used of Plummer distribution
                              NOTE: the above two values are chosen so
                                    that m( rfrac ) = mfrac                 [22.8042468]
seed             : Seed for the random number generator [0]
time             : Time at which snapshot is taken [0.0]
zerocm           : Centrate snapshot (t/f)? [t]
scale            : Model scale factor (-1=virial 1=natural) [-1]
quiet            : 0=noisy 1=somewhat quiet 2=more quiet [0]
massname         : If used Mass-function name (e.g. n(m)) []
massexpr         : Mass function expression (e.g. pow(m,p)) [pow(m,p)]
masspars         : Mass function parameters (e.g. p,0.0) [p,0.0]
massrange        : Range for mass-spectrum (e.g. 1,2) [1,1]
headlin

In [10]:
mkplummer --man

MKPLUMMER(1NEMO)                                              MKPLUMMER(1NEMO)

NAME
       mkplummer - construct a Plummer model

SYNOPSIS
       mkplummer [parameters=values...]

DESCRIPTION
       mkplummer  constructs  an N-body realization of a Plummer model, with a
       spatial or mass based cut-off, after which it (optionally)  performs  a
       coordinate  transformation to the center-of-mass coordinate system. The
       data are then written into a file snap-file, in a standard N-body snap‐
       shot(5NEMO)   format.    The  model  is  constructed  in  VIRIAL  units
       (M=G=-4E=1, with E the total energy), and finite spatial  extent  which
       can  be  regulated  by specifying mfrac or rfrac or using their default
       values.  The distribution function of a Plummer  model  is  spherically
       symmetric  and  isotropic, and is a polytrope of index n = 5.  See also
       Aarseth et al. (1974) and Plummer (1911).

       There is also an implementation in Dehn

In [11]:
mkplummer --help


mkplummer out=??? nbody=??? mlow=0 mfrac=0.999 rfrac=22.8042468 seed=0 time=0.0 zerocm=t scale=-1 quiet=0 massname= massexpr=pow(m,p) masspars=p,0.0 massrange=1,1 headline= nmodel=1 mode=1 VERSION=3.0b


## Initial conditions


In [12]:
mkplummer p128 128

The contents is binary.  So viewing contents needs a program **tsf** (*type structured file*)

In [13]:
tsf p128

char Headline[35] "init_xrandom: seed used 1706751345"
char History[32] "mkplummer p128 128 VERSION=3.0b"
set SnapShot
  set Parameters
    int Nobj 128 
    double Time 0.00000 
  tes
  set Particles
    int CoordSystem 66306 
    double Mass[128] 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      0.00781250 0.00781250 0.00781250 0.00781250 0.00781250 
      . . .
    double PhaseSpace[128][2][3] 0.398609 0.128806 -0.381117 -0.379700 
      -0.153210 -0.545662 0.0461589 -0.00924254 -0.342256 0.125321 
      -0.184185 0.607058 -0.471192 0.170905 -0.118584 -0.321103 
      -0.110769 -0.391427 1.03572 0.906998 -0.0721780 0.0139824 
      . . .
  tes
tes


## Integrate it:

The **hackcode1** program is the original Barnes and Hut (1985) tree code N-logN algorithm to integrate an N-body system.

In [14]:
hackcode1 p128 p128.out tstop=2 


init_xrandom: seed used 1706751345

       nbody        freq         eps         tol
         128       32.00      0.0500      1.0000

	options: mass,phase

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.000   -0.2588   -0.4961      6501        30        20      0.00

	        cm pos   -0.0000    0.0000   -0.0000
	        cm vel    0.0000   -0.0000    0.0000

	particle data written

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.031   -0.2590   -0.4957      6448        31        19      0.00

	        cm pos   -0.0000    0.0000   -0.0000
	        cm vel   -0.0001    0.0001   -0.0001

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.062   -0.2592   -0.4952      6685        33        18      0.00

	        cm pos   -0.0000    0.0000   -0.0000
	        cm vel   -0.0003    0.0001   -0.0002

        tnow       T+U       T/U     nttot     nbavg     ncavg   cputime
       0.094   -0.2597   

Display some diagnostics, such as energy and center of mass motion

In [15]:
snapdiagplot p128.out yapp=fig1.ps


### nemo Debug Info: 65 diagnostic frames read
Worst fractional energy loss dE/E = (E_t-E_0)/E_0 = -0.00556972 at T = 1.65625


This gives around 1% energy conservation, whichis not too bad, at least for galactic type simulations. For star clusters probably not too great at all.

Now plot in a two by two panel the X-Y projections of the integration at times 0, 1 and 2.

In [16]:
snapplot p128.out nxy=2,2 times=0,1,2 yapp=fig2.ps

## Compiling code

Although the build has done this, the **mknemo** command will recompile a program. This can be useful if the code needs to be edited for a bug fix and new feature. The **mknemo** command can be issued anywhere, and will show where the file in question lives. It can then easily be edited and recompiled, and added to $NEMOBIN. Except for editing, the **mknemo** command does all of this.

In [17]:
mknemo tsf

MKNEMO> Searching tsf.c: 
found one: /home/teuben/NEMO/demo/nemo_demo/src/kernel/io/tsf.c
gcc -g -O2 -D_FILE_OFFSET_BITS=64 -D_LARGEFILE_SOURCE -D_LARGEFILE64_SOURCE  -fpic -rdynamic  -Dlinux -DSYSV  -I/home/teuben/NEMO/demo/nemo_demo/inc -I/home/teuben/NEMO/demo/nemo_demo/lib -I/home/teuben/NEMO/demo/nemo_demo/opt/include    -Wall -Wimplicit-function-declaration  -D_GNU_SOURCE -std=c99  -o tsf tsf.c -L/home/teuben/NEMO/demo/nemo_demo/lib -L/home/teuben/NEMO/demo/nemo_demo/opt/lib          -lnemo -ldl    -lm   -L/usr/lib/gcc/x86_64-linux-gnu/11 -L/usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu -L/usr/lib/gcc/x86_64-linux-gnu/11/../../../../lib -L/lib/x86_64-linux-gnu -L/lib/../lib -L/usr/lib/x86_64-linux-gnu -L/usr/lib/../lib -L/usr/lib/gcc/x86_64-linux-gnu/11/../../.. -lgfortran -lm -lquadmath 



The output also reminds you wehre the **tsf.c** source code lives, so it can be easily put in the editor.


Here a program that uses graphics. These can be notoriously hard to link correctly.

In [18]:
mknemo tabplot

MKNEMO> Searching tabplot.c: 
found one: /home/teuben/NEMO/demo/nemo_demo/src/kernel/tab/tabplot.c
gcc -g -O2 -D_FILE_OFFSET_BITS=64 -D_LARGEFILE_SOURCE -D_LARGEFILE64_SOURCE  -fpic -rdynamic  -Dlinux -DSYSV  -I/home/teuben/NEMO/demo/nemo_demo/inc -I/home/teuben/NEMO/demo/nemo_demo/lib -I/home/teuben/NEMO/demo/nemo_demo/opt/include    -Wall -Wimplicit-function-declaration  -D_GNU_SOURCE -std=c99 -o tabplot tabplot.c \
	-L/home/teuben/NEMO/demo/nemo_demo/lib -L/home/teuben/NEMO/demo/nemo_demo/opt/lib          -lnemo -ldl    -lm  /home/teuben/NEMO/demo/nemo_demo/lib/yapp_ps.o   -lm



and a simple benchmark if graphics works in the **tabgen + tabplot** pipe:

In [19]:
tabgen - | tabplot - point=2,0.1 yapp=fig3.ps

### nemo Debug Info: read 10 points
### nemo Debug Info: min and max value in xcolumns 1: [0.011987 : 0.766750]
### nemo Debug Info: min and max value in ycolumns 2: [0.004308 : 0.988963]
### nemo Debug Info: X:min and max value reset to : [-0.025751 : 0.804488]
### nemo Debug Info: Y:min and max value reset to : [-0.044925 : 1.038196]


In [20]:
date1=$(date)
echo "Started at:   $date0"
echo "Done at:      $date1"

Started at:   Wed Jan 31 08:33:33 PM EST 2024
Done at:      Wed Jan 31 08:35:46 PM EST 2024


How did you do?  Did you get 298 executables in $NEMOBIN, did you install take about 2 minutes?